In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset #

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
dataset = pd.read_csv("../input/question-cpe-swu2019/question_cpe_swu2019.csv")
dataset

In [ ]:
Text_in_dataset = []
Class_in_dataset = []
for row in dataset.to_dict('records'):
    Text_in_dataset.append(row['question'])
    Class_in_dataset.append(row['class'])
#-------------------------------------------#
# print('Text: ',Text_in_dataset)    
# print('Class: ',Class_in_dataset)

# Preprocessing #

In [ ]:
!pip install --upgrade pythainlp

In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.util import dict_trie

In [ ]:
## custom dictionary
words ={'มิด','เทอม','ภาค','ฤดู','เมื่อ','ไหร่','ทะเบียน','มศว','วัน','ยัง','ไง','เกิน','กำหนด','ทุน','การ','ศึกษา','อยาก','เท่า','ค่า','สมุด','วิศวกรรมศาสตร์','สำนัก','จัด','ตั้ง','ขาด','เรียน',
        'คำถาม','หรือ','กิน','ซัมเมอร์', 'ตอน','วิชา','ปรับ','อะไร','ธนบุรี','มหาวิทยาลัย','นเรศวร','บูรพา','มหิดล','สอบ','กลาง','ไหน','จะ','เดือน','ไม่','ได้','จ่าย','ต้อง','เพิ่ม','ลด','ราย',
        'ขอ','ถ้า','หาก','ล่า','ช้า','ทำ','ทัน','ติดต่อ','หนังสือ','หอพัก','คืน','ไป','ที่','โดน','ตัด','ห้อง','คือ','จอง','คณะ','ศูนย์','ของ','ผิด','ถอน','หมา','เกรด','โปร','ดี','ครับ','สวัสดี',
        'ค่ะ','ครับ','บ้าง','คอมพิวเตอร์','วิศวะ','กำลัง','อาจารย์','ราม','เทคโนโลยี','พระจอมเกล้า','ลาดกระบัง','พระนครเหนือ','องค์','นอก','องครักษ์','วิทย์','เว็บไซต์','มี','ไหม','นวัตกรรม','พัก',
        'ทราบ','อยู่','ธรรมศาสตร์','ศิลปากร','จุฬา','รังสิต', 'โครงการ','วิทยาการ','แตก', 'ต่าง', 'การบ้าน','ข้อสอบ','นี้','กับ','คำร้อง','หิว','ข้าว','ติด','ลอย','หน่อย','อย่างไร','ถาม','ทั่วไป',
        'เป็น','อากาศ','ป่วย','ใบ','ข้อความ','บันทึก','คู่มือ','นิสิต'}
custom_dictionary_trie = dict_trie(words)

In [ ]:
#"Word Tokenize"
List_Text_token = []
print("##-----Word Tokenize-----##")
for t in Text_in_dataset :
    token = word_tokenize(t, engine="newmm" , custom_dict=custom_dictionary_trie)
    print("ประโยคที่ถูกตัดคำแล้ว:",token)
    print("จำนวนคำ:",len(token))
    List_Text_token.append(token)
    print("-----------------")
print("จำนวนประโยคทั้งหมด: ",len(List_Text_token))
    

In [ ]:
## Limit  word /text
#เติมให้ครบ 10 คำ
maxseq = 10
Text_preparation = []
for s in List_Text_token :
    n = maxseq - len(s)
    if n > 0:
        Text_preparation.append(s+(["<EOS>"]*n))#เติมให้ครบ ตามที่ Limit คำ
    elif n < 0:
        Text_preparation.append(s[0:maxseq]) #ตัดให้เหลือ Limit คำ
    else:
        Text_preparation.append(s)
# print('dataset_preparation:',Text_preparation)

**Word2Vec**

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
model_WordVec = KeyedVectors.load_word2vec_format('../input/word-vectors-th-from-fasttext/cc.th.300.vec',binary=False)

In [ ]:
XX = np.zeros((len(Text_preparation), 10, 300))
for i in range(len(Text_preparation)):
    for j in range(len(Text_preparation[i])):
        if Text_preparation[i][j] in model_WordVec.index_to_key:
            XX[i,j] = model_WordVec.word_vec(Text_preparation[i][j])
        else :
            pass

In [ ]:
# Splitting the data train/test
X_train = XX[:700]
Y_train = Class_in_dataset[:700]
X_test  = XX[700:]
Y_test  = Class_in_dataset[700:]
# validation
X_val = X_train[-70:]
Y_val = Y_train[-70:]
X_train = X_train[:-70]
Y_train = Y_train[:-70]
#-----------------------#
print(f"จำนวนข้อมูล X_train/X_val/X_test: {len(X_train)}/{len(X_val)}/{len(X_test)}")
print(f"จำนวนข้อมูล Y_train/Y_val/Y_test: {len(Y_train)}/{len(Y_val)}/{len(Y_test)}")

In [ ]:
# list to numpy array
Y_test = np.array(Y_test)
Y_val  = np.array(Y_val)
Y_train = np.array(Y_train)

# Define Model #

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
        layers.Bidirectional(layers.GRU(15),input_shape=(10,300)),
        layers.Dense(7, activation='softmax'),
])

# Add Optimizer, Loss, and Metric #

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy',]
)
model.summary()

# Train #

In [ ]:
history = model.fit( X_train,Y_train ,
                     validation_data = (X_val,Y_val),
                     epochs = 50,
                     batch_size = 16,)

# Evaluate #

In [ ]:
results = model.evaluate(X_test, Y_test)
print("Loss is", results[0])
print("Accuracy score is", results[1])